Alexandre CARRE*, Marvin LEROUSSEAU, Enzo BATTISTELLA <img align="right" width="400" height="40" src="../images/epu_ia_logo.png"> <br> *alexandre.carre@gustaveroussy.fr (Notebook conception) <br> marvin.lerousseau@gustaveroussy.fr (Notebook revision) <br> enzo.battistella@gustaveroussy.fr (Notebook revision)

# TP1: IMAGING DATAS PREPROCESSING

In this notebook you will learn how to process brain images before extracting data from it. First, download necessary materials for the afternoon practical sessions.

In [1]:
!git clone https://github.com/Alxaline/EPU-IA-2021.git
%cd EPU-IA-2021/

Cloning into 'TP2'...
remote: Invalid username or password.
fatal: Authentication failed for 'https://github.com/EPU-IA-2020/TP2.git/'
[Errno 2] No such file or directory: 'TP2/'
/media/acarre/Data/PythonProjects/TP2/version_google_colab


Ensure GPU is enabled in Google Colaboratory for further processing (otherwise, enabling GPU later will reboot the running session):

In [2]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found. Enable GPU in Google colaboratory: "Modifier" -> "Paramètres du notebook" -> Sélectionner "GPU" dans "Accélerateur matériel" puis relancer le notebook.')
print('Found GPU at: {}. All good!'.format(device_name))

UsageError: Line magic function `%tensorflow_version` not found.


### Data Description

Gliomas are primary brain tumours that start in glial cells. Gliomas can be classified into different histopathologic grades according to World Health Oraganization (WHO) grading system which represents malignancy. Gliomas can be low-grade "LGG" (slow-growing) or high-grade "HGG" (fast-growing).
It is important to differentiate HGG from LGG for assessing tumor progression and therapy planning (Louis et al., 2007).
The notebooks aim is to present key concepts to predict the grades of Gliomas using Radiomics imaging features.
Radiomics is a method capable of comprehensive quantification of tumor phenotypes by extracting a large number of quantitative imaging features.This quantitative analysis method can characterize tumor property in a non-invasive manner and it also can be used as a powerful tool to find biomarker that predict the diagnosis and prognosis of disease along with other clinical parameters. 

This dataset comes from a Multimodal Brain Tumor Segmentation Challenge which started in 2012 and is still running today.
 
All BraTS multimodal scans are available as NIfTI files (.nii.gz) and describe a) T1 and b) post-contrast T1-weighted (T1Gd), c) T2-weighted (T2), and d) T2 Fluid Attenuated Inversion Recovery (T2-FLAIR) volumes, and were acquired with different clinical protocols and various scanners from multiple (n=19) institutions.

All the imaging datasets have been segmented manually, by one to four raters, following the same annotation protocol, and their annotations were approved by experienced neuro-radiologists. Annotations comprise the Gd-enhancing tumor (ET — label 4), the peritumoral edema (ED — label 2), and the necrotic and non-enhancing tumor core (NCR/NET — label 1), as described both in the BraTS 2012-2013 TMI paper (1) and in the latest BraTS summarizing paper (2). The provided data are distributed after their pre-processing, i.e. **co-registered to the same anatomical template, interpolated to the same resolution (1 mm^3) and skull-stripped**.

Following radiological assessment of both the Glioblastoma Multiforme (TCGA-GBM, n=262) and the Low-Grade-Glioma (TCGA-LGG, n=199) collections, Bakas, Spyridon, et al. (3) identified 135 and 108 pre-operative mMRI scans, respectively. This is the numbers of samples that we will used for machine learning. For this part, you will have acces to 10 patients with LGG and 10 patients with GBM. 

### Installation of python dependencies

In [ ]:
!pip install SimpleITK numpy matplotlib

## Step 1: Explore images of the Dataset

In this step, the goal will be to visualize the different MRI imaging sequences associated with the tumor segmentation.

### 1) Load the fifth types of images available

- T1
- T1ce
- T2
- flair
- seg 

In [ ]:
import os
import glob
import SimpleITK as sitk
import numpy as np


def read_nii(img_path):
    """
    Function to read a NIfTI image from a filepath and return a nD array.
    :param img_path: file path of the image to be read
    :return: nD array
    """
    return sitk.GetArrayFromImage(sitk.ReadImage(img_path)) # sitk: z,y,x


# function to read one subject data
def create_data_one_subject(src, count, image_type):
    """
    Function to read one subject data from the general input directory specifying by the count number and image_type
    :param src: general input directory of all data
    :param count: patient to be read (same order as the glob function walk in the path)
    :param image_type: specify the image type to be read, can be 't1ce', 't1', 'flair', 't2', 'seg'
    :return: a dict for the 'seg' with key as type of seg and value as nD array. For image a nD array
    """
    # add check range of count (depend of numbers of samples)
    files = glob.glob(os.path.join(src, '**/*{}.nii.gz'.format(image_type)), recursive=True)
    if not files:
        raise FileNotFoundError('No .nii.gz data were found in %s' % os.path.join(os.path.join(src, '**')))
    k = (len(files) - count) - 1
    file = files[k]
    print('Processing---', os.path.basename(file))

    if any(x in image_type.lower() for x in ['t1ce', 't1', 'flair', 't2']):
        return read_nii(file)

    if 'seg' in image_type.lower():
        label_full, label_nec, label_core, label_et = [], [], [], []
        for label_num in [1, 2, 4, 5]:
            img = read_nii(file)
            if label_num == 5:
                img[img != 0] = 1  # Region 1 => 1+2+3+4 complete tumor
                label_full = img
            if label_num == 1:
                img[img != 1] = 0  # only left necrosis
                label_nec = img
            if label_num == 2:
                img[img == 2] = 0  # turn edema to 0
                img[img != 0] = 1  # only keep necrosis, ET, NET = Tumor core
                label_core = img
            if label_num == 4:
                img[img != 4] = 0  # only left ET
                img[img == 4] = 1
                label_et = img
        return {'label_full': label_full, 'label_nec': label_nec,
                'label_core': label_core, 'label_et': label_et}

**Exercice:** From functions above <code>create_data_one_subject</code>, read all types of images ('t1ce', 't1', 'flair', 't2', 'seg'). The variable name will have the same name as the image type. The source directory for src parameters is '/data/preprocessed/'. The count which allow you to select the patient need a range value between 0 and 20..

In [ ]:
src = './data/preprocessed/'

## Write your code Here
t1 = 
t1ce =
t2 = 
flair = 
seg = 

In [ ]:
#@title Solution {display-mode: "form"}
%load solutions_exercices/read_one_subject.py

### 3) Choose the orientation view and the slice number

In [ ]:
slice_number = 105
view = 'axial' # can be 'coronal', sagittal, 'axial'

### 4) View Datas

In [ ]:
import matplotlib.pyplot as plt

if view == 'axial':
    axial = int(slice_number)
    coronal = slice(None)
    sagittal = slice(None)
if view == 'coronal':
    axial = slice(None)
    coronal = int(slice_number)
    sagittal = slice(None)
if view == 'sagittal':
    axial = slice(None)
    coronal = slice(None)
    sagittal = int(slice_number)  


plt.figure(figsize=(15,10))

plt.subplot(241)
plt.title('T1')
plt.axis('off')
plt.imshow(t1[axial, coronal, sagittal],cmap='gray')

plt.subplot(242)
plt.title('T2')
plt.axis('off')
plt.imshow(t2[axial, coronal, sagittal],cmap='gray')
    
plt.subplot(243)
plt.title('Flair')
plt.axis('off')
plt.imshow(flair[axial, coronal, sagittal],cmap='gray')

plt.subplot(244)
plt.title('T1ce')
plt.axis('off')
plt.imshow(t1ce[axial, coronal, sagittal],cmap='gray')

plt.subplot(245)
plt.title('Ground Truth (full)')
plt.axis('off')
plt.imshow(seg['label_full'][axial, coronal, sagittal],cmap='gray')

plt.subplot(246)
plt.title('Ground Truth (nec)')
plt.axis('off')
plt.imshow(seg['label_nec'][axial, coronal, sagittal],cmap='gray')

plt.subplot(247)
plt.title('Ground Truth (core)')
plt.axis('off')
plt.imshow(seg['label_core'][axial, coronal, sagittal],cmap='gray')

plt.subplot(248)
plt.title('Ground Truth (et)')
plt.axis('off')
plt.imshow(seg['label_et'][axial, coronal, sagittal],cmap='gray')

plt.tight_layout()
plt.show();

## Step 2: Images preprocessing

As mentionned above, all images in the BratS dataset have already gone through some pre-processing steps.
To show the different preprocessing steps, we will use a raw data of BRATs and then:
- Correct magnetic bias field
- Resample to 1mm x 1mm x 1mm
- Skull-strip data
- Normalize data

###  Installation of needed package

#### Installation of Ants

ANTs is popularly considered as a state-of-the-art medical image registration and segmentation toolkit (4). This library is coded in C++, this is why we will install it as a binary file. So, we won't use a python Wrapper and directly execute the command line with the !

1) Run the following command to download the binary of ANTs.

In [ ]:
%%bash
function gdrive_download () {
 CONFIRM=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate "https://docs.google.com/uc?export=download&id=$1" -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')
 wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$CONFIRM&id=$1" -O $2
 rm -rf /tmp/cookies.txt
}
gdrive_download 1R0e4r9XmEQxTj_RGw8rpOsyQuqW_4Ubj ANTs-28-03-2019.7z

2) Extract it using 7z, then copy the entire contents of the newly created bin folder to /usr/local/bin/, and finally test the installation by running _which_ command that should output the full path to antsRegistration such as _/usr/local/bin/antsRegistration_

In [ ]:
!7z x ANTs-28-03-2019.7z
!cp bin/* /usr/local/bin
!which antsRegistration

#### Installation of HD-BET

HD-BET is a brain extraction tool based on deep learning (5).

In [ ]:
!git clone https://github.com/MIC-DKFZ/HD-BET

In [ ]:
import os
install_path = os.path.join(os.getcwd(), 'HD-BET')

In [ ]:
!pip install -e "$install_path"

### 1) Load a sample of Raw Data

In [ ]:
raw_data_path = './data/raw/GBM/TCGA-02-0027/TCGA-02-0027_GADO.nii.gz'
raw_data = read_nii(raw_data_path)

### 2) Bias field correction

Bias field signal is a low-frequency and very smooth signal that corrupts MRI images specially those produced by old MRI (Magnetic Resonance Imaging) machines. Image processing algorithms such as segmentation, texture analysis or classification that use the graylevel values of image pixels will not produce satisfactory results. A pre-processing step is needed to correct for the bias field signal before submitting corrupted MRI images to such algorithms (6). The version used here is a variant of the popular nonparametric nonuniform intensity normalization (N3) algorithm proposed for bias field correction (N4ITK) (7).

In [ ]:
bias_field_correct_image_path = os.path.join(os.path.dirname(raw_data_path), os.path.splitext(os.path.splitext(os.path.basename(raw_data_path))[0])[0] + '_n4.nii.gz')
bias_field_path = os.path.join(os.path.dirname(raw_data_path), os.path.splitext(os.path.splitext(os.path.basename(raw_data_path))[0])[0] + '_bias_field.nii.gz')

!N4BiasFieldCorrection -d 3 -i  "$raw_data_path" -s 3 -c [10x10x10x10, 1e-7] -b [200] -o ["$bias_field_correct_image_path", "$bias_field_path"]  -v

#### Display datas

In [ ]:
# Load datas
raw_data = read_nii(raw_data_path)
bias_field_correct_image = read_nii(bias_field_correct_image_path)
bias_field = read_nii(bias_field_path)
image_difference = raw_data - bias_field_correct_image

plt.figure(figsize=(15,10))

plt.subplot(141)
plt.title('Raw image')
plt.axis('off')
plt.imshow(raw_data[80, :, :],cmap='gray')

plt.subplot(142)
plt.title('Bias field')
plt.axis('off')
plt.imshow(bias_field[80, :, :],cmap='gray')

plt.subplot(143)
plt.title('Bias field corrected image')
plt.axis('off')
plt.imshow(bias_field_correct_image[80, :, :],cmap='gray')

plt.subplot(144)
plt.title('Image difference')
plt.axis('off')
plt.imshow(image_difference[80, :, :],cmap='gray')

plt.show();

### 3) Resampling

Voxel size resampling is a vital preprocessing step for datasets that have variable voxel sizes. Specifically, isotropic voxel size is required for some texture features extraction. Down-sampling to larger voxel dimensions will lead to information loss. On the contrary, up-sampling may add artifical information. It is still unclear which one is better and perhaps the best way to evaluate this is to base the judgement on the model's performances (8).

In [ ]:
resampled_image_path_1mm = os.path.join(os.path.dirname(raw_data_path), os.path.splitext(os.path.splitext(os.path.basename(raw_data_path))[0])[0] + '_n4_resampled_1mm.nii.gz')
resampled_image_path_2mm = os.path.join(os.path.dirname(raw_data_path), os.path.splitext(os.path.splitext(os.path.basename(raw_data_path))[0])[0] + '_n4_resampled_2mm.nii.gz')
resampled_image_path_6mm = os.path.join(os.path.dirname(raw_data_path), os.path.splitext(os.path.splitext(os.path.basename(raw_data_path))[0])[0] + '_n4_resampled_6mm.nii.gz')

!ResampleImage 3 "$bias_field_correct_image_path" "$resampled_image_path_1mm" 1x1x1 0 4 
!ResampleImage 3 "$bias_field_correct_image_path" "$resampled_image_path_2mm" 2x2x2 0 4 
!ResampleImage 3 "$bias_field_correct_image_path" "$resampled_image_path_6mm" 6x6x6 0 4 

#### Display datas

In [ ]:
# Load datas
raw_data = read_nii(raw_data_path)
resampled_image_1mm = read_nii(resampled_image_path_1mm)
resampled_image_2mm = read_nii(resampled_image_path_2mm)
resampled_image_6mm = read_nii(resampled_image_path_6mm)


plt.figure(figsize=(15,10))

plt.subplot(141)
plt.title('Raw image')
plt.axis('off')
plt.imshow(raw_data[80, :, :],cmap='gray')

plt.subplot(142)
plt.title('Bias field corrected + \n Resampled 1 mm')
plt.axis('off')
plt.imshow(resampled_image_1mm[120, :, :],cmap='gray')

plt.subplot(143)
plt.title('Bias field corrected +  \n Resampled 2 mm')
plt.axis('off')
plt.imshow(resampled_image_2mm[60, :, :],cmap='gray')

plt.subplot(144)
plt.title('Bias field corrected +  \n Resampled 6 mm')
plt.axis('off')
plt.imshow(resampled_image_6mm[20, :, :],cmap='gray')

plt.show();

### 4) Skull-Stripping

The skull stripping method is an important area of research in brain image processing applications. It acts as a preliminary step in numerous medical applications as it increases speed and accuracy of diagnosis in manifold. It removes non-cerebral tissues like skull, scalp, and dura from brain images (9). It will allow to remove skull where the range of intensities variation is high and base the subsequent normalization only on brain tissues.

In [ ]:
skull_stripped_image_path = os.path.join(os.path.dirname(raw_data_path), os.path.splitext(os.path.splitext(os.path.basename(raw_data_path))[0])[0] + '_n4_resampled_1mm_ss.nii.gz')

!hd-bet -i "$resampled_image_path_1mm" -o "$skull_stripped_image_path"

#### Display datas

In [ ]:
# Load datas
raw_data = read_nii(raw_data_path)
resampled_image_1mm = read_nii(resampled_image_path_1mm)
resampled_image_1mm_ss = read_nii(skull_stripped_image_path)

plt.figure(figsize=(15,10))

plt.subplot(131)
plt.title('Raw image')
plt.axis('off')
plt.imshow(raw_data[80, :, :],cmap='gray')

plt.subplot(132)
plt.title('Bias field corrected + \n Resampled 1 mm')
plt.axis('off')
plt.imshow(resampled_image_1mm[120, :, :],cmap='gray')

plt.subplot(133)
plt.title('Bias field corrected +  \n Resampled 1 mm + Skull-stripped')
plt.axis('off')
plt.imshow(resampled_image_1mm_ss[120, :, :],cmap='gray')


plt.show();

### 5) Normalization

Intensity normalization is an important preprocessing step in brain magnetic resonance image (MRI) analysis. During MR image acquisition, different scanners or parameters would be used for scanning different subjects or the same subject at a different time, which may result in large intensity variations. This intensity variation will greatly undermine the performance of subsequent MRI processing and population analysis, such as image registration, segmentation, and tissue volume measurement (10).

The method used here is a simple method: Z-score normalization uses the brain mask B corresponding to the image I to determine the mean and standard deviation of the intensities inside the brain mask, that is: 
$$ \mu = \frac{1}{|B|} \sum_{\mathbf b \in B} I(\mathbf b) \quad \text{and} \quad
\sigma = \sqrt{\frac{\sum_{\mathbf b \in B} (I(\mathbf b) - \mu)^2}{|B|-1}} $$
Then the Z-score normalized image is
$$ I_{\text{z-score}}(\mathbf x) = \frac{I(\mathbf x) - \mu}{\sigma}. $$</p>

In [ ]:
import SimpleITK as sitk

def zscore_normalize(img_path, mask=None):
    """
    Function to Z-Score normalize an image from an image filepath. It will normalize the target 
    image by subtracting the mean of the whole brain and dividing by the standard deviation.
    :param img_path: target MR brain image path
    :param mask: brain mask path for img
    :return: Normalized image in sitk format
    """
    img = sitk.ReadImage(img_path)
    img_data = sitk.GetArrayFromImage(img)
    origin, direction, spacing = img.GetOrigin(), img.GetDirection(), img.GetSpacing()
    if mask is not None and isinstance(mask, str):
        mask_data = read_nii(mask)
    elif mask == 'nomask':
        mask_data = img_data == img_data
    else:
        mask_data = img_data > img_data.mean()
    logical_mask = mask_data == 1  # force the mask to be logical type
    mean = img_data[logical_mask].mean()
    std = img_data[logical_mask].std()
    normalized_data = sitk.GetImageFromArray((img_data - mean) / std)
    normalized_data.SetOrigin(origin)
    normalized_data.SetDirection(direction)
    normalized_data.SetSpacing(spacing)
    return normalized_data

In [ ]:
img_path = skull_stripped_image_path
mask_path = os.path.join(os.path.dirname(raw_data_path), os.path.splitext(os.path.splitext(os.path.basename(raw_data_path))[0])[0] + '_n4_resampled_1mm_ss_mask.nii.gz')

normalized_image = zscore_normalize(img_path=skull_stripped_image_path)

normalized_image_path = os.path.join(os.path.dirname(raw_data_path), os.path.splitext(os.path.splitext(os.path.basename(raw_data_path))[0])[0] + '_n4_resampled_1mm_ss_normalize.nii.gz')

# write image
sitk.WriteImage(normalized_image, normalized_image_path)

#### Display datas

In [ ]:
# Load datas
raw_data = read_nii(raw_data_path)
resampled_image_1mm = read_nii(resampled_image_path_1mm)
resampled_image_1mm_ss = read_nii(skull_stripped_image_path)

plt.figure(figsize=(15,10))

plt.subplot(141)
plt.title('Raw image')
plt.axis('off')
plt.imshow(raw_data[80, :, :],cmap='gray')

plt.subplot(142)
plt.title('Bias field corrected + \n Resampled 1 mm')
plt.axis('off')
plt.imshow(resampled_image_1mm[120, :, :],cmap='gray')

plt.subplot(143)
plt.title('Bias field corrected +  \n Resampled 1 mm + Skull-stripped')
plt.axis('off')
plt.imshow(resampled_image_1mm_ss[120, :, :],cmap='gray')

plt.subplot(144)
plt.title('Bias field corrected +  \n Resampled 1 mm + only skull')
plt.axis('off')
plt.imshow(resampled_image_1mm[120, :, :] - resampled_image_1mm_ss[120, :, :],cmap='gray')


plt.show();

# References
(1): Menze, Bjoern H., et al. "The multimodal brain tumor image segmentation benchmark (BRATS)." IEEE transactions on medical imaging 34.10 (2014): 1993-2024.

(2): Bakas, Spyridon, et al. "Identifying the best machine learning algorithms for brain tumor segmentation, progression assessment, and overall survival prediction in the BRATS challenge." arXiv preprint arXiv:1811.02629 (2018).

(3): Bakas, Spyridon, et al. "Advancing the cancer genome atlas glioma MRI collections with expert segmentation labels and radiomic features." Scientific data 4 (2017): 170117.

(4): https://github.com/ANTsX/ANTs

(5): Isensee, Fabian, et al. "Automated brain extraction of multi-sequence MRI using artificial neural networks." arXiv preprint arXiv:1901.11341 (2019).

(6): Juntu, Jaber, et al. "Bias field correction for MRI images." Computer Recognition Systems. Springer, Berlin, Heidelberg, 2005. 543-551.

(7): Tustison, Nicholas J., et al. "N4ITK: improved N3 bias correction." IEEE transactions on medical imaging 29.6 (2010): 1310.

(8): Li, Ruijiang, et al., eds. Radiomics and Radiogenomics: Technical Basis and Clinical Applications. CRC Press, 2019.

(9): Roy, Shaswati, and Pradipta Maji. "A simple skull stripping algorithm for brain MRI." 2015 Eighth International Conference on Advances in Pattern Recognition (ICAPR). IEEE, 2015.

(10): Sun, Xiaofei, et al. "Histogram-based normalization technique on human brain magnetic resonance images from different acquisitions." Biomedical engineering online 14.1 (2015): 73.